In [ ]:
import pandas as pd
import os
from selenium import webdriver
import time

In [ ]:
# ensure version of chromedriver matches with chrome version
# location of chromedriver.exe
f = open("chrome_driver_location.txt")
driver_path = f.read()

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--incognito")

In [ ]:
# go to website 
driver = webdriver.Chrome(driver_path,options=options)

In [ ]:
link = # the first page of google maps search result
driver.get(link)

In [ ]:
# scrape location names and addresses
names, addresses = [],[]

def try_until_succeed(element_num):
    # try clicking on element name until successful
    # https://stackoverflow.com/questions/18225997/stale-element-reference-element-is-not-attached-to-the-page-document
    
    status = "fail"
    num_tries = 0
    while (status == "fail") and (num_tries<=5):
        try:
            num_tries += 1
            
            s = driver.find_element_by_xpath(f"//div[@data-result-index='{element_num}']/div/div/div/div/h3")
            s.click()
            status = "success"
        except:
            status = "fail"
            

end = False
page = 1
while end is False:
    print(f"At page {page}")
    
    # determine how many elements in one page 
    all_sections = driver.find_elements_by_xpath("//h3[@class='section-result-title']")
    n = len(all_sections)

    for element_num in range(1,n+1):
        
        # get element name
        name =  driver.find_element_by_xpath(f"//div[@data-result-index='{str(element_num)}']/div/div/div/div/h3").text
        names.append(name)
        
        # click on element to go to element's page 
        try_until_succeed(element_num)
        time.sleep(5) # wait for element's page to load
        
        # once element's page is already loaded....
        try:
            address = driver.find_element_by_xpath("//div[@data-section-id='ad']").text
        except:
            address = None
        addresses.append(address)
        time.sleep(3)
        
        # go back to search page
        back=driver.find_element_by_xpath("//div[@class='section-layout section-layout-root']/button")
        back.click()
        time.sleep(5)
        
    # when we have finished scraping all results in one search page
    try:
        # go to next page 
        next_page = driver.find_element_by_xpath("//div[@class='gm2-caption']/div/div/button[@aria-label=' Next page ']")
        next_page.click()
    except: # no more next page button 
        end = True
    page += 1


In [ ]:
df = pd.DataFrame({"name": names, "address": addresses})